In [1]:
import pybullet
import gym
import numpy as np
import matplotlib.pyplot as plt
import pybullet_data
import math
import rrtstar
import mpc
import torch

pybullet build time: Feb  8 2022 11:09:50


In [ ]:
#!git clone https://github.com/ROBOTIS-GIT/turtlebot3.git

In [2]:
# Initializing pybullet
physicsClient = pybullet.connect(pybullet.GUI)
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())
pybullet.setGravity(0,0,-9.8)

# Loading chessgrid plane
planeID = pybullet.loadURDF('plane.urdf')
chess_grid = pybullet.loadTexture('checker_huge.gif')
pybullet.changeVisualShape(planeID, -1, textureUniqueId=chess_grid)




# Creating robots
player = pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.25,0.25,0], pybullet.getQuaternionFromEuler([0,0,0]))

start = (0.25,0.25)

static = []

static.append(pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.75,0.25,0], pybullet.getQuaternionFromEuler([0,0,math.pi])))
static.append(pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.75,0.75,0], pybullet.getQuaternionFromEuler([0,0,math.pi])))
static.append(pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.75,-0.25,0], pybullet.getQuaternionFromEuler([0,0,math.pi])))
static.append(pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [1.25,0.75,0], pybullet.getQuaternionFromEuler([0,0,math.pi])))
target = static[3]

goal = (1.25, 0.75)

# For milestone 2 only - setting all non-player targets to have no collision with player robot
# Doesn't work when dragging into each other but might still work?
pybullet.setCollisionFilterGroupMask(player, target, 0, 0)

for robot in static:
    pybullet.setCollisionFilterGroupMask(player, robot, 0, 0)

# Setting colors to distinguish robots. BLUE for player, GREEN for obstacle, RED for target
pybullet.changeVisualShape(player, 0, rgbaColor=[0,0,1,1])
for robot in static:
    pybullet.changeVisualShape(robot, 0, rgbaColor=[0,1,0,1])
pybullet.changeVisualShape(target, 0, rgbaColor=[1,0,0,1])



# Setting up overhead camera
viewMatrix = pybullet.computeViewMatrix(
            cameraEyePosition=[0.75,0.25,3],
            cameraTargetPosition=[0.75,0.25,0],
            cameraUpVector=[0,1,0])

projectionMatrix = pybullet.computeProjectionMatrixFOV(
            fov=45.0,
            aspect=1.0,
            nearVal=0.1,
            farVal=3.1)

width, height, rgbImg, depthImg, segImg = pybullet.getCameraImage(width = 224, height = 224, viewMatrix=viewMatrix, projectionMatrix = projectionMatrix)

# Getting position and orientation
print(pybullet.getBasePositionAndOrientation(player))
print(pybullet.getBasePositionAndOrientation(static[0]))
print(pybullet.getBasePositionAndOrientation(static[1]))
print(pybullet.getBasePositionAndOrientation(static[2]))
print(pybullet.getBasePositionAndOrientation(target))


# Start simulation
pybullet.setRealTimeSimulation(1)

Version = 4.1 ATI-4.7.103
Vendor = ATI Technologies Inc.
Renderer = AMD Radeon Pro 5300M OpenGL Engine
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: base_footprint
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: imu_link
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagona

In [3]:
# RRT
samplingArea = [0.25, 1.25, -0.25, 0.75]
RRT = rrtstar.RRT(start, goal, [], samplingArea, 'rrtstar', 'rectangle', 2)
path = RRT.planning()

[0.25, 0.25, 0]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[1.1349923006472582, 0.6342366003446572, 2.419342217065119]
[0. 0. 0.]
[1.1349923006472582, 0.6342366003446572, 2.419342217065119]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[1.1349923006472582, 0.6342366003446572, 2.419342217065119]
[0. 0. 0.]
[0.3159522147081518, 0.09859605906072078, 6.214077384731243]
[0. 0. 0.]
[0.3159522147081518, 0.09859605906072078, 6.214077384731243]
[0. 0. 0.]
[0.3159522147081518, 0.09859605906072078, 6.214077384731243]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[1.1349923006472582, 0.6342366003446572, 2.419342217065119]
[0. 0. 0.]
[0.3159522147081518, 0.09859605906072078, 6.214077384731243]
[0. 0. 0.]
[0.8125066318429058, 0.38042452178864483, 6.0819330221434855]
[0. 0. 0.]
[0.8125066318429058, 0.38042452178864483, 6.0819330221434855]
[0. 0. 0.]
[0.8125066318429058, 0.38042452178864483, 6.0819330221434855]
[0. 0. 0.]
[1.1349923006472582, 0.6342366003446572, 2.419342217065119]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[1.

In [4]:
print(path)

[[1.25, 0.75, 0], [1.25, 0.75, 0], [0.7263514878618315, 0.4828909759131923, 0.24798179134910855], [0.25, 0.25, 0]]


In [ ]:
# MPC
horizon_length = 5
opt_iters = 10
samples = 100
top_samples = 10

device = torch.device('cpu')

In [5]:
pybullet.disconnect()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
